# 概要説明

これは、特定のロジックをテストするためだけに完全な Azure 関数を実行するという厳密さを伴わずに、Azure 関数のコンポーネントを簡単にテストするためのノートブックです。これは単体テストに代わるものではなく、Azure 関数を使用して手動で最新の状態に保つ必要がありますが、ロジックのユニットをすばやく試すのに便利です。

In [1]:
%pip install -r ../src/requirements.txt
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langdetect import detect_langs
import logging
import os
import json
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [3]:
from dotenv import load_dotenv

load_dotenv()

ai_multiservice_endpoint = os.environ.get("AI_MULTISERVICE_ENDPOINT")
ai_multiservice_key = os.environ.get("AI_MULTISERVICE_KEY")

print("Using Azure AI multi-service endpoint: ", ai_multiservice_endpoint)
print("Using Azure AI multi-service key: ", len(ai_multiservice_key) * "*")

Using Azure AI multi-service endpoint:  https://cajetzeraiservices.cognitiveservices.azure.com/
Using Azure AI multi-service key:  ********************************


## ユーティリティ関数

### validate_and_format_json
C# の TryParse スタイルのメソッドと同様の方法で使用され、オブジェクトが JSON として解析可能かどうかをテストし、操作結果を示すフラグと結果の JSON または null 値の両方を含むタプルを提供します。JSONに対して有効な'''null'''値を取り戻す可能性がある場合、通常はブール値の値が関係してきますが、この場合、それは機能的に同じであり、この関数の使用意図であるOpen AIからのありそうもない結果であるため、JSONまたは'''null'''に減らすことができます。これは、Open AIがプロンプトエンジニアリングに基づいてJSON応答を生成したことを検証します。

In [5]:
def validate_and_format_json(json_string, indent=4):
    try:
        # Attempt to parse the JSON string
        parsed_json = json.loads(json_string)
        # If successful, re-encode it with indentation for pretty printing
        return True, json.dumps(parsed_json, indent=indent)
    except json.JSONDecodeError:
        # If parsing fails, return an error message
        return False, None

### parse_text_to_boolean
'''validate_and_format_json''' と同様に、これは Open AI 出力の検証を容易にすることを目的としています。この場合、ブール応答を取得しようとし、有効な応答が得られた場合はそれを提供し、それ以外の場合は null 値を提供します。これは、ブール応答のみに簡略化することもできますが、ダウンストリームのニーズに対する理解が不十分なため、ブール値の解析の失敗は否定的なテスト結果とは異なるものとして扱います。

In [6]:
def parse_text_to_boolean(text):
    if text.lower() == "true":
        return True
    elif text.lower() == "false":
        return False
    else:
        return None

#### calculate_mode_response
これは、比較可能なJSONオブジェクトのリストを取得し、すべてのオブジェクトのプロパティでモード値を見つけようとする、より複雑なユーティリティ関数です。現在、わかりやすくするために、標準のスキーマを前提としています。現在、この関数で取り組んでいる2つの「バグ」があります
1. リスト プロパティはハッシュ可能ではないため、モード応答を識別するために使用される応答の一時的なディクショナリのキーとして使用することはできません。リストの応答をフラット化し、リスト内の各値に対してそれらを応答に追加する必要があります
2. この関数は現在、空の JSON オブジェクトを効果的に処理していません
私は、これが主にPythonコードとの親密さの欠如のためにパフォーマンスを発揮しないのではないかと懸念しています。これはこの問題に取り組むための合理的な方法だと思いますが、論理的な観点からより創造的でパフォーマンスの高い解決を考え出す人を想像することもできます。

In [7]:
# def calculate_mode_response(list_of_json_objects):
#     print(list_of_json_objects)

#     if len(list_of_json_objects) == 0:

#         return None

#     # remove any None objects from the list
#     list_of_json_objects = [val for val in list_of_json_objects if val is not None]

#     first_entry = json.loads(list_of_json_objects[0])

#     property_keys = first_entry.keys()

#     mode_response = {}

#     responses = {}

#     for key in property_keys:

#         for obj in list_of_json_objects:
#             if isinstance(obj, list):
#                 print(f"{obj} is a list in {list_of_json_objects}", list_of_json_objects)

#             json_obj = json.loads(obj)

#             value = json_obj[key]

#             # todo: determine how to handle list values, for now skipping to prepare for demo

#             if isinstance(value, list):

#                 continue

#             if json_obj[key] in responses:

#                 responses[value] += 1

#             else:

#                 responses[value] = 1

#         if len(responses) > 0:

#             mode_response[key] = max(responses, key=responses.get)

#         responses = {}

#     return mode_response

### flatten_json
テストを試行します。

In [13]:
def flatten_json(json_obj):
    out = {}

    def flatten(x, name=""):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + "_")
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + "_")
                i += 1
        else:
            out[name[:-1]] = x

    flatten(json_obj)
    return out

## チャンキング
チャンキングのテクニックについては、chunking-research.ipynb で詳しく説明しています。ここでは、フローの残りの部分をテストする際に活用できるチャンクを取得しようとしています。 *注目すべき変更* ここで、それを関数に変え、uri_pathをファイルパスに変更したことです。

In [7]:
def chunk_document(file_path):
    ai_doc_intel_loader = AzureAIDocumentIntelligenceLoader(
        file_path=file_path,
        api_key=ai_multiservice_key,
        api_endpoint=ai_multiservice_endpoint,
        api_model="prebuilt-layout",
    )

    docs = ai_doc_intel_loader.load()

    # Split the document into chunks base on markdown headers.
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    docs_string = docs[0].page_content

    splits = text_splitter.split_text(docs_string)

    chunks = []
    
    for split in splits:
        chunks.append(split.page_content)

    return chunks

## イントロダクションとエンフォースメントチャンクの特定
これは、より多くの作業と実験が必要となる領域です。マークダウンの周りで、AI関連ではない面倒なコーディングを行うことができるかもしれません。私は、各チャンクを1つの文に要約し、チャンクインデックスで文に番号を付け、AOAIにどの文が要約に属しているかを推測させるという、さまざまな結果を持つ手法を試しました。それは、より洗練されたプロンプトで機能できると思います。今のところ、より素朴なソリューションで実行しています

In [10]:
def identify_introduction_chunks(chunks):
    intro_chunks = []
    intro_chunks.append(chunks[0])  
    return intro_chunks

def identify_conclusion_chunks(chunks):
    conclusion_chunks = []
    conclusion_chunks.append(chunks[-1])
    return conclusion_chunks

## セマンティックカーネルのセットアップ
....
- プラグインディレクトリを変更して、プロジェクト内のプラグインコードから実際に借用し、ノートブックと機能の間のパリティを維持することを容易にする他のコードの一部でこれを試みることができますが、コードを並べて簡単に説明できる利点が失われます...

todo: functionalize and return kernel

```__file__``` Jupyter Notebook では利用できないため、os.getcwd を使用します

In [11]:
curr_directory = os.getcwd()
plugins_directory = os.path.join(curr_directory, "..\\src\\plugins")

# setup semantic kernel
aoai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
aoai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
aoai_key = os.environ.get("AZURE_OPENAI_API_KEY")

print(f"Using aoai_deployment: {aoai_deployment}")
print(f"Using aoai_endpoint: {aoai_endpoint}")
print(f"Using aoai_key: {len(aoai_endpoint)*'*'}")

kernel = sk.Kernel()

service_id = "default"

service = AzureChatCompletion(
    service_id=service_id,
    deployment_name=aoai_deployment,
    endpoint=aoai_endpoint,
    api_key=aoai_key,
)

kernel.add_service(service)

plugin_names = [
    plugin
    for plugin in os.listdir(plugins_directory)
    if os.path.isdir(os.path.join(plugins_directory, plugin))
]

# for each plugin, add the plugin to the kernel
try:
    for plugin_name in plugin_names:
        kernel.import_plugin_from_prompt_directory(plugins_directory, plugin_name)
except ValueError as e:
    logging.exception(f"Plugin {plugin_name} not found")

Using aoai_deployment: gpt-35-turbo-16k-deployment
Using aoai_endpoint: https://exp-aoai.openai.azure.com/
Using aoai_key: **********************************


## ドキュメントの分類
これは、導入と結論を評価するだけでコストを節約しながら、比較的正確な結果を得ることができると信じています。時折、正確な結果が得られることがあります。大きなドキュメントの場合は、チャンクを結合できない可能性があるため、各チャンクに対してプラグインを呼び出し、結果のモードを見つけるように修正する必要があるかもしれません。

In [12]:
async def classify_study_type(chunks, kernel):
    joined_chunks = f"{os.linesep}".join(chunks)
    skresult = await kernel.invoke(
        kernel.plugins["ClassificationPlugin"]["ClassifyStudyType"],
        sk.KernelArguments(input=joined_chunks,tense="present"),
    )
    _, intro_study_type_classification = validate_and_format_json(
        skresult.value[0].content, None
    )
    return intro_study_type_classification

## センチメント分析
ここでは、調査結果が決定的で重要であるかどうかを判断しようと試みますが、これも導入チャンクを使用して実行できると信じています。分類と同様に、チャンクが大きすぎて大きなドキュメントには対応できない場合は、チャンクごとに実行し、結果を平均化する必要があるかもしれません。

In [13]:
async def are_findings_conclusive_and_signficant(chunks, kernel):
    joined_chunks = f"{os.linesep}".join(chunks)
    skresult = await kernel.invoke(
        kernel.plugins["SummaryPlugin"]["AreStudyFindingsSignificant"],
        sk.KernelArguments(input=joined_chunks),
    )
    return parse_text_to_boolean(skresult.value[0].content)

## エンティティの抽出
ここからが忙しくなるところです。ステークホルダー、日付、そしてすべてのチャンクに対して実行しているマルチエンティティ抽出関数があり、可能性を見逃さないようにし、結果のモードを見つけます

In [20]:
def calculate_mode_response(list_of_json_objects, printObjs=False):
    if len(list_of_json_objects) == 0:
        return None

    # remove any None objects from the list
    list_of_json_objects = [val for val in list_of_json_objects if val is not None]

    if printObjs:
        print("List of JSON Objects: ", list_of_json_objects)

    first_entry = list_of_json_objects[0]
    # hacky conversion of other types to dictionary for keys
    schema = (
        json.loads(first_entry)
        if isinstance(first_entry, str)
        else json.loads(json.dumps(first_entry))
    )

    if printObjs:
        print("Schema: ", schema)

    property_keys = schema.keys()

    mode_response = {}
    responses = {}

    for key in property_keys:
        if printObjs:
            print("Beginning loop for key: ", key)
        for obj in list_of_json_objects:
            if printObjs:
                print("Evaluating: ", obj)

            # hacky conversion of other types to dictionary for keysl see above
            json_obj = (
                json.loads(obj) if isinstance(obj, str) else json.loads(json.dumps(obj))
            )

            flatten_json(json_obj)
            if isinstance(obj, list):
                print(
                    f"{obj} is a list in {list_of_json_objects}", list_of_json_objects
                )

            value = json_obj[key]

            # todo: determine how to handle list values, for now skipping to prepare for demo

            if isinstance(value, list):
                continue

            if json_obj[key] in responses:
                responses[value] += 1
            else:
                responses[value] = 1

        if len(responses) > 0:
            mode_response[key] = max(responses, key=responses.get)
        responses = {}
    return mode_response


# for lists like this ['[{"a": "b"}, {"c": "d"}]', '[{"e": "f"}, {"g": "h"}]'] -> [{"a": "b"}, {"c": "d"}, {"e": "f"}, {"g": "h"}]
def flatten_list_of_list_strings(original_list):
    list_of_lists = [json.loads(val) for val in original_list if val is not None]
    return [val for sublist in list_of_lists for val in sublist]


###
async def use_extraction_function(chunk, function_name, kernel):
    extract_entities_result = await kernel.invoke(
        kernel.plugins["EntityExtraction"][function_name],
        sk.KernelArguments(input=chunk),
    )
    # print(extract_entities_result.value[0].content)
    _, extracted_entities = validate_and_format_json(
        extract_entities_result.value[0].content, None
    )

    return extracted_entities



async def extract_entities(chunks, kernel):
    extracted_entity_responses = []
    extracted_stakeholders_responses = []
    extracted_dates_responses = []

    for chunk in chunks:
        extract_entities = await use_extraction_function(
            chunk, "ExtractMultipleEntities", kernel
        )
        extracted_entity_responses.append(extract_entities)
        extracted_stakeholders = await use_extraction_function(
            chunk, "ExtractStakeholders", kernel
        )
        extracted_stakeholders_responses.append(extracted_stakeholders)
        extracted_dates = await use_extraction_function(
            chunk, "ExtractSignificantDates", kernel
        )
        extracted_dates_responses.append(extracted_dates)


    # mode entity extraction results across chunks

    # multi entities
    mode_extracted_entities_response = calculate_mode_response(
        extracted_entity_responses
    )

    # stakeholders
    extracted_stakeholders_responses = flatten_list_of_list_strings(
        extracted_stakeholders_responses
    )
    mode_extracted_stakeholders_response = calculate_mode_response(
        extracted_stakeholders_responses
    )

    # dates
    extracted_dates_responses = flatten_list_of_list_strings(extracted_dates_responses)
    mode_extracted_dates_response = calculate_mode_response(
        extracted_dates_responses
    )


    deep_final_result = {
        "avg_extracted_entities_response": mode_extracted_entities_response,
        "avg_extracted_stakeholders_response": mode_extracted_stakeholders_response,
        "avg_extracted_dates_response": mode_extracted_dates_response,
    }


    flat_final_result = flatten_json(deep_final_result)

    return flat_final_result

## フローをつなぐ

In [23]:
# todo: read document names from sample docs directory and then loop through

curr_directory = os.getcwd()
sample_docs_directory = os.path.join(curr_directory, "..\\sample-docs")
dir_list = os.listdir(sample_docs_directory)

file_path = os.path.join(sample_docs_directory, dir_list[0])
print(f"Operating against document: {file_path}")

chunks = chunk_document(file_path)
intro_chunks = identify_introduction_chunks(chunks)
conclusion_chunks = identify_conclusion_chunks(chunks)

combined_intro_conclusion_chunks = intro_chunks + conclusion_chunks

study_type = await classify_study_type(combined_intro_conclusion_chunks, kernel)

are_findings_significant = await are_findings_conclusive_and_signficant(
    combined_intro_conclusion_chunks, kernel
)  # potentially just use conclusion chunks

# right now, we are just using the first chunk to determine the language, depending on business case you could use all chunks in the event it is a multi-language document you'll be more likely to get a more accurate result
languages_result = detect_langs(intro_chunks[0])
languages = []
for lang in languages_result:
    languages.append({"language": lang.lang, "probability": lang.prob})

entities = await extract_entities(chunks, kernel)

deep_final_result = {
    "entities": entities,
    "languages": languages,
    "are_findings_significant": are_findings_significant,
    "study_type": study_type,
}

flat_final_result = flatten_json(deep_final_result)

print(json.dumps(flat_final_result, indent=4))

Operating against document: c:\Projects\kagami\notebooks\..\sample-docs\2 - jbm-5-015.pdf
Processing chunk 0
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
{
    "entities_avg_extracted_entities_response_Duration": "",
    "entities_avg_extracted_entities_response_DrugOrCompound": "",
    "entities_avg_extracted_entities_response_RouteOfAdministration": "",
    "entities_avg_extracted_entities_response_InternalStudyNumber": "",
    "entities_avg_extracted_entities_response_ExternalStudyNumber": "",
    "entities_avg_extracted_entities_response_TestFacility": "",
    "entities_avg_extracted_stakeholders_response_name": "methylprednisolone",
    "entities_avg_extracted_

# check_for_handwritten_signature
Document Intelligence を使用して、ドキュメントのチャンク内の手書きの署名を検出する機能をテストする

In [18]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
curr_directory = os.getcwd()
sample_docs_directory = os.path.join(curr_directory, "..\\sample-docs")
dir_list = os.listdir(sample_docs_directory)

file_path = os.path.join(sample_docs_directory, dir_list[0])
print(f"Operating against document: {file_path}")

# Replace 'ai_multiservice_key' and 'ai_multiservice_endpoint' with your actual credentials

def check_for_handwritten_style(file_path):
    # Extract the text chunk from the arguments
    chunks = chunk_document(file_path)

    # Use the Document Intelligence API to analyze the text in the last chunk
 
    text_chunk = chunks[-1]
    # api_key = os.environ.get("AZURE_FORM_RECOGNIZER_API_KEY")
    # endpoint = os.environ.get("AZURE_FORM_RECOGNIZER_ENDPOINT")
    credential = AzureKeyCredential(ai_multiservice_key)
    client = DocumentIntelligenceClient(endpoint=ai_multiservice_endpoint, credential=credential)
    with open(file_path, "rb") as f:
        poller = client.begin_analyze_document(
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream"
        )
    result: AnalyzeResult = poller.result()

    if result.styles and any([style.is_handwritten for style in result.styles]):
        return True
    else: return False
        # result = client.begin_analyze_document(text_chunk).result()

    # Check if the text is handwritten style
    # is_handwritten_style = result.isHandwritten()
    # return is_handwritten_style

has_handwritten_sig = check_for_handwritten_style(file_path)
print("has_handwritten_sig = " + str(has_handwritten_sig))

Operating against document: c:\GitHub\kagami\notebooks\..\sample-docs\DLM_Window_Quote_Jan21_signed.pdf
has_handwritten_sig = True
